In [4]:
import sys
sys.path.append("mrcnn")
from m_rcnn import *
import cv2
from visualize import *
import requests
import json
import numpy as np
from PIL import Image
import io
import base64
import warnings
from flask import Flask, request, Response
from flask import send_file
import imutils
import glob
import tensorflow as tf
import keras
from keras import backend as K

warnings.filterwarnings('ignore')
def encode(val):
    val=val.replace("+","BGABG")
    val=val.replace("=","AGAAGA")
    val=val.replace("/","OPAWQQQQ")
    return val
def decode(val):
    val=val.replace("BGABG","+")
    val=val.replace("AGAAGA","=")
    val=val.replace("OPAWQQQQ","/")
    return val
def stringToImage(base64_string):
    imgdata = base64.b64decode(base64_string)
    return Image.open(io.BytesIO(imgdata))
# convert PIL Image to an RGB image( technically a numpy array ) that's compatible with opencv
def toRGB(image):
    return cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)


app=Flask(__name__)

@app.route('/')
def index():
    return "Knee instance segmentation"

@app.route('/knee')
def get_json(methods=["GET"]):
    sess = K.get_session()
    test_model, inference_config =load_inference_model(1,"mask_rcnn_object_0005.h5")
    input_url=request.args.get('image',type=str)
    img_string=decode(input_url)
    img = toRGB(stringToImage(img_string))
    transparent=np.zeros_like(img)
    height,width,c=img.shape
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h,w,c=img.shape
    
    r = test_model.detect(np.array(image).reshape(-1,h,w,c))[0]
    colors = random_colors(80)
    object_count = len(r["class_ids"])
    for i in range(object_count):
        mask = r["masks"][:, :, i]
        contours = get_mask_contours(mask)
        for cnt in contours:
            cv2.polylines(img, [cnt], True, colors[i], 2)#only the outlines
            cv2.polylines(transparent, [cnt], True, colors[i], 1)#only the outlines
    _,buffer=cv2.imencode('.jpg',transparent)
    img_encoded=base64.b64encode(buffer)
    img_st=encode((str(img_encoded)[2:])[:-1])
    my_dict=dict()
    my_dict['img_st']=img_st
    K.clear_session()
    return my_dict 





app.run()
'''imgs=glob.glob('imgs//*.jpg')
for i in imgs:
    img=cv2.imread(i)
    h,w,c=img.shape
    if w>500:
        img=imutils.resize(img,width=500)
    _,buffer=cv2.imencode('.jpg',img)
    img_encoded=base64.b64encode(buffer)
    sttt=(str(img_encoded)[2:])[:-1]
    e=encode(sttt)

    img_trans=decode(get_json(e)["img_st"])

    img_transparent = toRGB(stringToImage(img_trans))

    cv2.imshow('img',cv2.add(img,img_transparent))
    cv2.waitKey(0)
    cv2.destroyAllWindows()'''


ModuleNotFoundError: No module named 'm_rcnn'